# Find the highest-scoring local alignment between two strings

CMSC701-0101 Spring 2019 Computational Genomics   
University of Maryland   
Reginald Carey   
for Dr. Mihai Pop

### Local Alignment Problem

Find the highest-scoring local alignment between two strings.

**Given:** Two amino acid strings.

**Return:** The maximum score of a local alignment of the strings, followed by a local alignment of these strings achieving the maximum score. Use the PAM250 scoring matrix and indel penalty σ = 5. (If multiple local alignments achieving the maximum score exist, you may return any one.)

#### Sample Dataset
```
MEANLY
PENALTY
```

#### Sample Output
```
15
EANL-Y
ENALTY
```

**Extra Dataset**   
[Click for an extra dataset](http://bioinformaticsalgorithms.com/data/extradatasets/alignment/local_alignment.txt)

### The Smith-Waterman Algorithm

In [1]:
import sys
import numpy as np
from collections import Counter
import re
from collections import defaultdict
from functools import partial

In [2]:
def score_function(score_name="BASIC", A=1, B=0, C=1):

    def score(a, b, indel, s):
        """Return the match/mismatch/indel score for the symbols a, and b."""
        return indel if '-' in [a,b] else s[a][b]
    
    if "BLOSUM62" == score_name:
        inputString = """
               A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  W  Y
            A  4  0 -2 -1 -2  0 -2 -1 -1 -1 -1 -2 -1 -1 -1  1  0  0 -3 -2
            C  0  9 -3 -4 -2 -3 -3 -1 -3 -1 -1 -3 -3 -3 -3 -1 -1 -1 -2 -2
            D -2 -3  6  2 -3 -1 -1 -3 -1 -4 -3  1 -1  0 -2  0 -1 -3 -4 -3
            E -1 -4  2  5 -3 -2  0 -3  1 -3 -2  0 -1  2  0  0 -1 -2 -3 -2
            F -2 -2 -3 -3  6 -3 -1  0 -3  0  0 -3 -4 -3 -3 -2 -2 -1  1  3
            G  0 -3 -1 -2 -3  6 -2 -4 -2 -4 -3  0 -2 -2 -2  0 -2 -3 -2 -3
            H -2 -3 -1  0 -1 -2  8 -3 -1 -3 -2  1 -2  0  0 -1 -2 -3 -2  2
            I -1 -1 -3 -3  0 -4 -3  4 -3  2  1 -3 -3 -3 -3 -2 -1  3 -3 -1
            K -1 -3 -1  1 -3 -2 -1 -3  5 -2 -1  0 -1  1  2  0 -1 -2 -3 -2
            L -1 -1 -4 -3  0 -4 -3  2 -2  4  2 -3 -3 -2 -2 -2 -1  1 -2 -1
            M -1 -1 -3 -2  0 -3 -2  1 -1  2  5 -2 -2  0 -1 -1 -1  1 -1 -1
            N -2 -3  1  0 -3  0  1 -3  0 -3 -2  6 -2  0  0  1  0 -3 -4 -2
            P -1 -3 -1 -1 -4 -2 -2 -3 -1 -3 -2 -2  7 -1 -2 -1 -1 -2 -4 -3
            Q -1 -3  0  2 -3 -2  0 -3  1 -2  0  0 -1  5  1  0 -1 -2 -2 -1
            R -1 -3 -2  0 -3 -2  0 -3  2 -2 -1  0 -2  1  5 -1 -1 -3 -3 -2
            S  1 -1  0  0 -2  0 -1 -2  0 -2 -1  1 -1  0 -1  4  1 -2 -3 -2
            T  0 -1 -1 -1 -2 -2 -2 -1 -1 -1 -1  0 -1 -1 -1  1  5  0 -2 -2
            V  0 -1 -3 -2 -1 -3 -3  3 -2  1  1 -3 -2 -2 -3 -2  0  4 -3 -1
            W -3 -2 -4 -3  1 -2 -2 -3 -3 -2 -1 -4 -4 -2 -3 -3 -2 -3 11  2
            Y -2 -2 -3 -2  3 -3  2 -1 -2 -1 -1 -2 -3 -1 -2 -2 -2 -1  2  7
            """
    elif "PAM250" == score_name:
        inputString = """
               A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  W  Y
            A  2 -2  0  0 -3  1 -1 -1 -1 -2 -1  0  1  0 -2  1  1  0 -6 -3
            C -2 12 -5 -5 -4 -3 -3 -2 -5 -6 -5 -4 -3 -5 -4  0 -2 -2 -8  0
            D  0 -5  4  3 -6  1  1 -2  0 -4 -3  2 -1  2 -1  0  0 -2 -7 -4
            E  0 -5  3  4 -5  0  1 -2  0 -3 -2  1 -1  2 -1  0  0 -2 -7 -4
            F -3 -4 -6 -5  9 -5 -2  1 -5  2  0 -3 -5 -5 -4 -3 -3 -1  0  7
            G  1 -3  1  0 -5  5 -2 -3 -2 -4 -3  0  0 -1 -3  1  0 -1 -7 -5
            H -1 -3  1  1 -2 -2  6 -2  0 -2 -2  2  0  3  2 -1 -1 -2 -3  0
            I -1 -2 -2 -2  1 -3 -2  5 -2  2  2 -2 -2 -2 -2 -1  0  4 -5 -1
            K -1 -5  0  0 -5 -2  0 -2  5 -3  0  1 -1  1  3  0  0 -2 -3 -4
            L -2 -6 -4 -3  2 -4 -2  2 -3  6  4 -3 -3 -2 -3 -3 -2  2 -2 -1
            M -1 -5 -3 -2  0 -3 -2  2  0  4  6 -2 -2 -1  0 -2 -1  2 -4 -2
            N  0 -4  2  1 -3  0  2 -2  1 -3 -2  2  0  1  0  1  0 -2 -4 -2
            P  1 -3 -1 -1 -5  0  0 -2 -1 -3 -2  0  6  0  0  1  0 -1 -6 -5
            Q  0 -5  2  2 -5 -1  3 -2  1 -2 -1  1  0  4  1 -1 -1 -2 -5 -4
            R -2 -4 -1 -1 -4 -3  2 -2  3 -3  0  0  0  1  6  0 -1 -2  2 -4
            S  1  0  0  0 -3  1 -1 -1  0 -3 -2  1  1 -1  0  2  1 -1 -2 -3
            T  1 -2  0  0 -3  0 -1  0  0 -2 -1  0  0 -1 -1  1  3  0 -5 -3
            V  0 -2 -2 -2 -1 -1 -2  4 -2  2  2 -2 -1 -2 -2 -1  0  4 -6 -2
            W -6 -8 -7 -7  0 -7 -3 -5 -3 -2 -4 -4 -6 -5  2 -2 -5 -6 17  0
            Y -3  0 -4 -4  7 -5  0 -1 -4 -1 -2 -2 -5 -4 -4 -3 -3 -2  0 10
            """
    elif "BASIC" == score_name:
        return lambda a, b: -C if '-' in [a,b] else A if a == b else -B
    else:
        raise ValueError("score_name constrained to ['BLOSUM62', 'PAM250']")

    lines = inputString.strip().splitlines()
    amino_acids = "".join(re.split(" +", lines[0].strip()))
    target = defaultdict(dict)
    for row in lines[1:]:
        rowId, *values = re.split(" +", row.strip())
        for colId, value in zip(amino_acids, values):
            target[rowId][colId] = int(value)

    return partial(score, indel=-C, s=target)

In [3]:
def _sequence_similarity(X, Y, s, global_seq):
    """
    Compute the sequence similarity between to sequences X and Y.  We return a populated matrix representing
    the confusion between the two sequences as well as the optimal alignment data.
    
    This is our implementation of the Needleman-Wunsch Algorithm.
    
    """
    def maximum(*args):
        m = max(map(lambda sv: sv[1], args))
        s = "".join(map(lambda sv: sv[0], filter(lambda sv: sv[1] == m, args)))
        return (s, m)
    def horzValue(i, j):
        """Return the value of the horizontal path."""
        return mat[i, j-1][1] + s('-', X[j-1])
    def diagValue(i, j):
        """Return the value of the diagonal path."""
        return mat[i-1, j-1][1] + s(Y[i-1], X[j-1])
    def vertValue(i, j):
        """Return the value of the vertical path."""
        return mat[i-1, j][1] + s(Y[i-1], '-')

    m = len(Y) + 1
    n = len(X) + 1
    index = (-1, -1)
    maxval = -sys.maxsize
    minval = -sys.maxsize * global_seq
    mat = np.empty((m, n), dtype=tuple)
    for i in range(0, m):
        mat[i, 0] = maximum(("", minval), ("v", i * s('-', '-')))
        maxval, index = (mat[i, 0][1], (i, 0)) if mat[i, 0][1] > maxval else (maxval, index)
    for j in range(0, n):
        mat[0, j] = maximum(("", minval), ("h", j * s('-', '-')))
        maxval, index = (mat[0, j][1], (0, j)) if mat[0, j][1] > maxval else (maxval, index)
    for i in range(1, m):
        for j in range(1, n):
            mat[i, j] = maximum(("", minval), ("h", horzValue(i, j)), ("d", diagValue(i, j)), ("v", vertValue(i, j)))
            maxval, index = (mat[i, j][1], (i, j)) if mat[i, j][1] > maxval else (maxval, index)
    if global_seq:
        index = m-1, n-1
        maxval = mat[m-1, n-1][1]
    return (mat, maxval, index)

def global_sequence_similarity():
    return partial(_sequence_similarity, global_seq=True)

def local_sequence_similarity():
    return partial(_sequence_similarity, global_seq=False)

In [4]:
def global_traceback(mat, X, Y):
    """
    Traceback returns one of potentially many optimal alignments.  Each point where a cell has more than
    one maximal value, we have a bifurication leading to a doubling in the number of optimal alignments.
    """
    n, m = mat.shape[1]-1, mat.shape[0]-1
    retval = ("","")
    while m > 0 or n > 0:
        a, b = ("","")
        if 'h' in mat[m, n][0]:
            a, b = X[n-1], "-"
            n = n - 1
        elif 'd' in mat[m, n][0]:
            a, b = X[n-1], Y[m-1]
            n, m = n - 1, m - 1
        elif 'v' in mat[m, n][0]:
            a, b = "-", Y[m-1]
            m = m - 1
        else:
            break
        retval = (a + retval[0], b + retval[1])
    return retval

In [5]:
def local_traceback(mat, X, Y):
    """
    Traceback returns one of potentially many optimal alignments.  Each point where a cell has more than
    one maximal value, we have a bifurication leading to a doubling in the number of optimal alignments.
    """
    n, m = mat.shape[1]-1, mat.shape[0]-1
    retval = ("","")
    while (m > 0 or n > 0) and mat[m, n][1] > 0:
        a, b = ("","")
        if 'h' in mat[m, n][0]:
            a, b = X[n-1], "-"
            n = n - 1
        elif 'd' in mat[m, n][0]:
            a, b = X[n-1], Y[m-1]
            n, m = n - 1, m - 1
        elif 'v' in mat[m, n][0]:
            a, b = "-", Y[m-1]
            m = m - 1
        else:
            break
        retval = (a + retval[0], b + retval[1])
    return retval

In [8]:
def assignment_3():
    [X, Y, *_] = open("/Users/ReginaldCarey/Downloads/rosalind_ba5f.txt").read().split("\n")
    similarity = local_sequence_similarity()
    mat,maxval,index = similarity(X, Y, s=score_function("PAM250", C=5))
    print(maxval)
    print("\n".join(local_traceback(mat[:index[0]+1, :index[1]+1], X, Y)))

In [9]:
assignment_3()

4061
STWMHPEIIIEALQEYSIELMKIVHRYGTNEKNLQIKEGKVFMNIWNYDFGIMS-CTMCVY-QPY--FEL-KSY-NRISDSM--MTW-D-TTWSWQ-NT-----PDNISNTI-RT-YFNPKGPTDASYAEFLAIFLRDNKKNLIKVSWDYPWQFWFIKIWCGGPLDKGYTCKHYRADWTWDDWQWNCCWIIA-AIANGHSGKNEFHYDDS-G-F-TWDW-QDFPYGASKTAE-SNG---D-I-C--WQPQK-MCKW-PVDCSNCCMCYDVGKWNNVKRYLHMP-YQG--SIMDQDSAELDSKIWYYDESNAMPDKESFMTDWEWGGCVYLIHWKTDRMEIHKEKYVDR--VISGCPETMDEFNLYAL-VFAGDTKVW--VMVQPQFER-SNPFWVVIDNQDRGYTQGSSRHTHKDPCEPFIVFFSLWIPGIYH--KKEKFGQAAGYYTKKPPSM--P-FNLCEHSWTNYDWF-VKVTGRYTCH--KT-THCWQYQ-PWFYNMNDNTKPK-SW-MARWHDEVMVPAIDHRQWQEVFWRCWWKKECDWWMHYKDTGERMYGHPLRHNHHWGWRTATEHQFCDNDANYIPSACHFTKYCSN-R-CQSARNEMYKDGFMMSRHKCCCGNANIDHWKTTLLWDQNIWQTKNKRKAENLHAHVIHQMRIKLDMCQRGFSHPWPPTI-MWAPQSPDKQKFMGLLIGKQQWG-FYWRGKGMGETGAGIQNGC-H-QP--Y--HWFRMF--CYYPMDHVRNCKCKHG-KIFMLMWTSLLQHHKCLNDIEIICWIYW-AF--P-NYA-ECG--APRE-G-RS-WLLFLMD--AKR--D-C---TY-R-HI-Q-LYKLAEAWNCYVAQDGAAQFWEEWWGRSKQF--EPRHQMA-YM-H-P-PG-PYPN-HFFGLEHQTLQWRRI-EWNMDWFRGPKFINVFLYMLFLCTWEICVPIHGWDHYIWATELYRIDLQQMRISFNRARTFNYYWGKRVFTWDCFLWHWS